# Data Wrangeling project
# What influences wine quality ?
January 2024 

By Māra and Ovi 

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

### Data acquisition

In [ ]:
# Loading the dataset
data = pd.read_csv('WineQT.csv')

In [ ]:
# Looking at the initial data 
data.head
# The Id coulumn has an iteresting end. 

In [ ]:
data.info()
# Only quality and Id are integers the rest are objects.
# However the Id strangely reaches 1597 in the 1142 row. 
# There are no null values :)

In [ ]:
data.describe().T
# Apart from the Id's range being higher than the amount of entries,
    # the .... are the most interesting 

### Data Cleaning 

In [ ]:
# Changing the fromat
data = data.astype({'quality': float, 'Id': float})

In [ ]:
# Checking for missing values
missing_values = data.isnull().sum()
print(missing_values)

In [ ]:
# Checking for duplicate rows
duplicate_rows = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

In [ ]:
# Checking unique values in Id column
unique_id = data['Id'].nunique()
print(f"Number of unique id: {unique_id}")


In [ ]:
# Dropping the 'Id' column 
data = data.drop(columns='Id')

In [ ]:
# Renaming columns
new_column_names = [
    'fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
    'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
    'ph', 'sulphates', 'alcohol', 'quality'
]

data.columns = new_column_names

Dealing with outliers 

In [ ]:
# Creating box plots for each feature to inspect outliers

plt.figure(figsize=(18, 10))
plt.subplots_adjust(hspace=0.5)

for i, column in enumerate(data.columns[:-1], 1):  
    # excluding 'quality' column
    plt.subplot(3, 4, i)
    sns.boxplot(x=data[column])
    plt.title(f'Box plot of {column}')
plt.show()

In [ ]:
# Investigation the outliers using the IQR method

# Calculating Q1, Q3, and IQR for each feature
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# Determine outliers using the IQR method
outliers_iqr = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)))

# Print the outliers based on IQR method
print("Outliers based on IQR method:")
print(outliers_iqr)

In [ ]:
# Handling Outliers by capping at the 1st and 99th percentiles
for column in data.columns[:-1]:  
    percentiles = data[column].quantile([0.01, 0.99]).values
    data[column] = np.clip(data[column], percentiles[0], percentiles[1])

Computing dummy variables 

In [ ]:
# Creating a new feature 'free_sulfur_dioxide_ratio'
data['free_sulfur_dioxide_ratio'] = data['free_sulfur_dioxide'] / data['total_sulfur_dioxide']

### Data Merging - this is missing fully 

In [ ]:
# ....

### Data Aggregation 
    This needs way more 

Also this makes sense if it goes after the corrolation matrix ?

- Data transformation 
- Grouping ...
- Preforme calculations on subsets of data 


In [ ]:
# Aggregating data by 'quality' and computing the mean for each feature
print("\nData aggregated by wine quality:")
quality_summary = data.groupby('quality').mean()
print(quality_summary)

In [ ]:
data_grouped = data.groupby('quality')[['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
    'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
    'ph', 'sulphates', 'alcohol', 'quality']].mean()

data_grouped

### Data Visualization

In [ ]:
# Check the target variable
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

sns.countplot(x=df['quality'], ax=axes[0])
for c in axes[0].containers:
        axes[0].bar_label(c, fmt='%d', padding=1)
sns.boxplot(x=df['quality'], )

plt.title(f'Distribution of Quality')
plt.tight_layout()
plt.show()

# There are not that many drinks with good quality and bad quality nost are anly a bit over average. 
# Thus indicating that most wine quality is around 5.5 

In [ ]:
# Correlation matrix
print("\nCorrelation Matrix:")
plt.figure(figsize=(12, 8))
corr_matrix = data.corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# The higest positive corrolation is with the alcohol (0.48), and 
# the lowest corrolation with volitile acidity(-0.41). 

# Nonetheless, sulphates and citric acid also seem to be corrolated with the quality at around 0.25 
# Possibly also the density and the total sulfur dioxide at -0.18

Unsure if there is a need to do all of the features so we can do the ones i selected  

The code is below yours for now 

In [ ]:
# Visualizing the distributions of the features

print("\nVisualizing the distributions of the features:")
plt.figure(figsize=(18, 10))
plt.subplots_adjust(hspace=0.5)
for i, column in enumerate(data.columns[:-1], 1):  # excluding 'quality' column
    plt.subplot(3, 4, i)
    sns.histplot(data[column], kde=True)
    plt.title(f'Distribution of {column}')
plt.show()# Quality Distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='quality', data=data, palette="viridis")
plt.title('Distribution of Wine Quality Ratings')
plt.show()

In [ ]:
# Checking the distribution of the intresting features 
interest = ['volatile acidity', 'citric acid', 'total sulfur dioxide', 'density', 'sulphates', 'alcohol', 'Id']

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 9))

axes = axes.flatten()

for i, col in enumerate(interest):
    sns.kdeplot(data=df[col], ax=axes[i])
    axes[i].set_title(f'Distribution of {col}')

for j in range(len(interest), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

This as well we can adjust 

In [ ]:
# Box plots for each feature against wine quality
print("\nBox plots for each feature against wine quality:")
for column in data.columns[:-1]:  # excluding 'quality' column
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='quality', y=column, data=data)
    plt.title(f'{column} vs Quality')
    plt.show()

### Statistical Analysis

In [ ]:
# ANOVA for 'alcohol', 'sulphates', and 'volatile_acidity'
anova_results = {}
for feature in ['alcohol', 'sulphates', 'volatile_acidity']:
    model = ols(f'{feature} ~ C(quality)', data=data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    anova_results[feature] = anova_table
    print(f"ANOVA results for {feature}:\n", anova_table, "\n")

In [ ]:
# Tukey's HSD Test for 'alcohol', 'sulphates', and 'volatile_acidity'
tukey_results = {}
for feature in ['alcohol', 'sulphates', 'volatile_acidity']:
    tukey = pairwise_tukeyhsd(endog=data[feature], groups=data['quality'], alpha=0.05)
    tukey_results[feature] = tukey
    print(f"Tukey's HSD test results for {feature}:\n", tukey.summary(), "\n")
